In [8]:
# default_exp preprocess.prepare_transaction_data

# Preparing transaction data for the customers in the scope 

> API details.

In [9]:
#hide
from nbdev.showdoc import *

In [10]:
#export
import os
import pandas as pd
from sample_project import config
from sample_project.helper import write_to_csv, read_from_csv
from fastcore.utils import store_attr
import numpy as np

In [11]:
#hide
import warnings
warnings.filterwarnings("ignore")

In [86]:
#export
class Transactional_Data:
    '''
    This class is built to create main dataset to be used in this project. Following steps below, the transaction dataset for
    clients in the scope is created:
    1. Merging three different datasets: "transaction","customer info" and "disp info" which shows customer and account matches
    2. Applying three different filters below to cover only customers in the scope:
        - Consider only transactions whose date in between <start date> and <end date> 
        - Consider only customers who have loans with more than <loan_amnt_thrsh> euros
        - Consider only customers from districts where there are more than <district_cnt_thrsh> customers 
        
    Args:
            trnx_dataset (Pandas DataFrame): The csv file path which has transaction dataset with at least these fields: "account_id","date"
            disp_dataset (Pandas DataFrame): The csv file path which disp dataset with at least these fields: "client_id","account_id"
            client_dataset (Pandas DataFrame): The csv file path which customer info dataset with at least these fields: "client_id","district_id"
            loan_dataset (Pandas DataFrame): The csv file path which loan dataset with at least these fields: "client_id","amount"
            loan_amnt_thrsh (integer): Loan amount threshold to be use to apply filter 2
            district_cnt_thrsh (integer): District count threshold to be used to apply filter 3
            start_date (integer): Start date threshold for transaction dataset to apply filter 1
            end_date (integer): End date threshold for transaction dataset to apply filter 1
            to_csv (boolean): If the returned dataframe is desired to be written into csv file 

    Return:
            main_data (pandas DataFrame): the transaction dataset for clients in the scope 
    '''
    
    def __init__(self,trnx_dataset=config.CSV_TRANSACTION, 
                 disp_dataset= config.CSV_DISP_INFO, 
                 client_dataset=config.CSV_CUST_INFO,
                 loan_dataset=config.CSV_LOAN,
                 loan_amnt_thrsh=1000,
                 district_cnt_thrsh = 110,
                 start_date=900000,
                 end_date=970000):

        store_attr()
       
    
    def create_data(self, apply_filters = True, to_csv=True):
        
        df_trnx = read_from_csv(self.trnx_dataset)
        self.df_disp = read_from_csv(self.disp_dataset)
        self.df_client = read_from_csv(self.client_dataset)
        self.df_loan = read_from_csv(self.loan_dataset)
        
        merged_data = (df_trnx.drop(["k_symbol","bank","account"],axis=1)
                              .merge(self.df_disp[["client_id","account_id"]], on="account_id",how="left")
                              .merge(self.df_client[["client_id","district_id"]],on="client_id",how="left")
                             ) 
        
        if apply_filters: 
            merged_data = self.applying_date_filter(merged_data)
            merged_data = self.applying_loan_amount_filter(merged_data)
            merged_data = self.applying_district_filter(merged_data)
        
        if to_csv:
            write_to_csv(df= merged_data, path = config.CSV_CUSTOMIZED_TRNX )
            
        return merged_data
         
    def applying_date_filter(self,df):
        
        return df[(df["date"] >= self.start_date)&(df["date"] <= self.end_date)]
    
    def applying_loan_amount_filter(self,df):
        
        df = df.merge(
                        (self.df_loan
                             .merge(self.df_disp[["client_id","account_id"]],on="account_id",how="left")
                             .groupby("client_id",as_index=False).amount.sum().rename(columns={'amount':'Total_Loan_Amount'})
                         ), on ="client_id", how="left"
                      )
        df["Total_Loan_Amount"] = df["Total_Loan_Amount"].fillna(0)
        
        return df[df["Total_Loan_Amount"] > self.loan_amnt_thrsh]
    
    def applying_district_filter(self,df):
        
        district_count = self.df_client.groupby("district_id",as_index=False).client_id.count().rename(columns={'client_id':'Num_Cust_in_District'})
        district_list = district_count[district_count["Num_Cust_in_District"]>self.district_cnt_thrsh]["district_id"].tolist()
        
        return df[df["district_id"].isin(district_list)]
    

## Create data automatically

In [87]:
#hide
tranx= Transactional_Data(loan_amnt_thrsh=0, district_cnt_thrsh = 0)
tranx_data = tranx.create_data(apply_filters=True,to_csv=True)

In [88]:
#hide
tranx_data

,trans_id,account_id,date,type,operation,amount,balance,client_id,district_id,Total_Loan_Amount
81,1548749,5270,930113,"""PRIJEM""","""VKLAD""",800.0,800.0,6367,44,79608.0
91,1548750,5270,930114,"""PRIJEM""","""PREVOD Z UCTU""",44749.0,45549.0,6367,44,79608.0
94,3393738,11265,930114,"""PRIJEM""","""VKLAD""",1000.0,1000.0,13845,15,52788.0
100,3122924,10364,930117,"""PRIJEM""","""VKLAD""",1100.0,1100.0,12754,55,21924.0
101,3122924,10364,930117,"""PRIJEM""","""VKLAD""",1100.0,1100.0,12755,55,21924.0
...,...,...,...,...,...,...,...,...,...,...
539017,2548927,8411,961231,"""VYDAJ""","""VYBER""",14.6,20259.5,10389,20,220620.0
539018,2592488,8564,961231,"""VYDAJ""","""VYBER""",14.6,44131.0,10563,68,76680.0
539019,2592488,8564,961231,"""VYDAJ""","""VYBER""",14.6,44131.0,10564,68,76680.0
539036,516795,1766,961231,"""VYDAJ""","""VYBER""",14.6,27107.3,2141,28,30060.0


## Check the steps 

In [89]:
#hide
tranx= Transactional_Data(loan_amnt_thrsh=0, district_cnt_thrsh = 0)
merged_data = tranx.create_data(apply_filters=False,to_csv=False)
merged_data

,trans_id,account_id,date,type,operation,amount,balance,client_id,district_id
0,695247,2378,930101,"""PRIJEM""","""VKLAD""",700.0,700.0,2873,16
1,171812,576,930101,"""PRIJEM""","""VKLAD""",900.0,900.0,692,74
2,171812,576,930101,"""PRIJEM""","""VKLAD""",900.0,900.0,693,74
3,207264,704,930101,"""PRIJEM""","""VKLAD""",1000.0,1000.0,844,22
4,207264,704,930101,"""PRIJEM""","""VKLAD""",1000.0,1000.0,845,22
...,...,...,...,...,...,...,...,...,...
1262620,3627616,2935,981231,"""PRIJEM""","""""",81.3,19544.9,3545,68
1262621,3625403,2869,981231,"""PRIJEM""","""""",60.2,14638.2,3468,68
1262622,3626683,2907,981231,"""PRIJEM""","""""",107.5,23453.0,3513,70
1262623,3626683,2907,981231,"""PRIJEM""","""""",107.5,23453.0,3514,70


### Date filter

In [90]:
#hide
date_filtered = tranx.applying_date_filter(merged_data)
date_filtered

,trans_id,account_id,date,type,operation,amount,balance,client_id,district_id
0,695247,2378,930101,"""PRIJEM""","""VKLAD""",700.0,700.0,2873,16
1,171812,576,930101,"""PRIJEM""","""VKLAD""",900.0,900.0,692,74
2,171812,576,930101,"""PRIJEM""","""VKLAD""",900.0,900.0,693,74
3,207264,704,930101,"""PRIJEM""","""VKLAD""",1000.0,1000.0,844,22
4,207264,704,930101,"""PRIJEM""","""VKLAD""",1000.0,1000.0,845,22
...,...,...,...,...,...,...,...,...,...
539043,516262,1765,961231,"""VYDAJ""","""VYBER""",14.6,19708.1,2139,18
539044,516262,1765,961231,"""VYDAJ""","""VYBER""",14.6,19708.1,2140,18
539045,520019,1775,961231,"""VYDAJ""","""VYBER""",14.6,15944.5,2152,17
539046,517894,1769,961231,"""VYDAJ""","""VYBER""",14.6,34679.4,2145,39


In [91]:
#hide
print("number_of_removed_rows:", len(merged_data) - len(date_filtered))

number_of_removed_rows: 723577


### Loan Amount Filter

In [92]:
#hide
loan_amount_filtered = tranx.applying_loan_amount_filter(merged_data)
loan_amount_filtered

,trans_id,account_id,date,type,operation,amount,balance,client_id,district_id,Total_Loan_Amount
81,1548749,5270,930113,"""PRIJEM""","""VKLAD""",800.0,800.0,6367,44,79608.0
91,1548750,5270,930114,"""PRIJEM""","""PREVOD Z UCTU""",44749.0,45549.0,6367,44,79608.0
94,3393738,11265,930114,"""PRIJEM""","""VKLAD""",1000.0,1000.0,13845,15,52788.0
100,3122924,10364,930117,"""PRIJEM""","""VKLAD""",1100.0,1100.0,12754,55,21924.0
101,3122924,10364,930117,"""PRIJEM""","""VKLAD""",1100.0,1100.0,12755,55,21924.0
...,...,...,...,...,...,...,...,...,...,...
1262579,3626751,2910,981231,"""PRIJEM""","""""",187.2,42249.6,3517,68,437460.0
1262585,3627577,2933,981231,"""PRIJEM""","""""",132.0,31046.4,3542,10,272520.0
1262586,3627577,2933,981231,"""PRIJEM""","""""",132.0,31046.4,3543,10,272520.0
1262597,3626847,2912,981231,"""PRIJEM""","""""",271.6,58694.2,3519,74,380160.0


In [93]:
#hide
print("number_of_removed_rows:", len(merged_data) - len(loan_amount_filtered))

number_of_removed_rows: 1028998


### District Filter

In [94]:
#hide
district_filtered = tranx.applying_district_filter(merged_data)
district_filtered

,trans_id,account_id,date,type,operation,amount,balance,client_id,district_id
0,695247,2378,930101,"""PRIJEM""","""VKLAD""",700.0,700.0,2873,16
1,171812,576,930101,"""PRIJEM""","""VKLAD""",900.0,900.0,692,74
2,171812,576,930101,"""PRIJEM""","""VKLAD""",900.0,900.0,693,74
3,207264,704,930101,"""PRIJEM""","""VKLAD""",1000.0,1000.0,844,22
4,207264,704,930101,"""PRIJEM""","""VKLAD""",1000.0,1000.0,845,22
...,...,...,...,...,...,...,...,...,...
1262620,3627616,2935,981231,"""PRIJEM""","""""",81.3,19544.9,3545,68
1262621,3625403,2869,981231,"""PRIJEM""","""""",60.2,14638.2,3468,68
1262622,3626683,2907,981231,"""PRIJEM""","""""",107.5,23453.0,3513,70
1262623,3626683,2907,981231,"""PRIJEM""","""""",107.5,23453.0,3514,70


In [95]:
#hide
print("number_of_removed_rows:", len(merged_data) - len(district_filtered))

number_of_removed_rows: 0
